<a href="https://colab.research.google.com/github/lunaB/AI-Experiment/blob/master/%ED%96%A5%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json

req = Request("https://www.fragrantica.com/awards.php?show_mode=json_nominations&category_id=635", headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
obj = json.loads(webpage)

In [3]:
obj.keys()

dict_keys(['id', 'name', 'slug', 'category_type', 'category_filters', 'user_restrictions', 'nominations'])

In [4]:
# 10개만
collect_num = 10
nominations = obj['nominations'][0:collect_num]
nominations_url = [i['url'] for i in nominations]
nominations_name = [i['name'] for i in nominations]
nominations_url

['/perfume/Calvin-Klein/CK-One-276.html',
 '/perfume/Tom-Ford/Tobacco-Vanille-1825.html',
 '/perfume/Hermes/Un-Jardin-Sur-Le-Nil-18.html',
 '/perfume/Serge-Lutens/Chergui-2762.html',
 '/perfume/Mugler/Mugler-Cologne-708.html',
 '/perfume/Bvlgari/Black-154.html',
 '/perfume/Tauer-Perfumes/02-L-Air-du-Desert-Marocain-4573.html',
 '/perfume/Diptyque/Philosykos-3865.html',
 '/perfume/Chanel/Coromandel-Eau-de-Parfum-41783.html',
 '/perfume/4711/4711-Original-Eau-de-Cologne-3256.html']

In [5]:
## test code
url = 'https://www.fragrantica.com/perfume/Calvin-Klein/CK-One-276.html'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')
accord = soup.select('#main-content .cell .grid-x .cell.accord-box .accord-bar')
print(accord[0].text)
reviews = soup.select("#all-reviews .cell .grid-x .cell.fragrance-review-box div[itemprop='reviewBody']")
print(reviews[0].text)

citrus
I bought it as a gift for my fiancé about two or three years ago, and he didn't like it, so I got this perfume. I think it's really cool, fresh, citrusy scent, but it still smells fruity. In summer, I like to wear it in the very heat, or at home, for example, when cleaning or after morning workouts and showers. I think it's a super everyday fragrance. There's nothing intrusive about it. Clean, easy to wear, absolutely unisex. Longevity is great, even on my skin, but especially on clothes.


In [6]:
database = []
base = 'https://www.fragrantica.com'
for i in range(len(nominations_url)):
    url = base + nominations_url[i]
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    accords = soup.select('#main-content .cell .grid-x .cell.accord-box .accord-bar')
    reviews = soup.select("#all-reviews .cell .grid-x .cell.fragrance-review-box div[itemprop='reviewBody']")

    database.append({
        'name': nominations_name[i],
        'accords': list(map(lambda x: x.text, accords)),
        'reviews': list(map(lambda x: x.text, reviews))
    })

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from nltk.tokenize import word_tokenize, sent_tokenize
# sent_tokenize(database[0]['reviews'][0])
# word_tokenize(database[0]['reviews'][0])

In [65]:
from gensim.models import FastText
data = list(map(lambda x: word_tokenize(x), database[0]['reviews']))
model = FastText(data, size=100, window=5, min_count=5, workers=4, sg=1)
model.save('fasttext')
model.wv.save_word2vec_format("fasttext_vis")

In [61]:
# loaded_model = FastText.load("fasttext")

In [66]:
!python -m gensim.scripts.word2vec2tensor --input fasttext_vis --output fasttext_vis

2021-04-10 19:32:47,564 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py --input fasttext_vis --output fasttext_vis
2021-04-10 19:32:47,564 - utils_any2vec - INFO - loading projection weights from fasttext_vis
2021-04-10 19:32:47,614 - utils_any2vec - INFO - loaded (504, 100) matrix from fasttext_vis
2021-04-10 19:32:47,658 - word2vec2tensor - INFO - 2D tensor file saved to fasttext_vis_tensor.tsv
2021-04-10 19:32:47,658 - word2vec2tensor - INFO - Tensor metadata file saved to fasttext_vis_metadata.tsv
2021-04-10 19:32:47,658 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [67]:
model.wv.most_similar('good')

[('looking', 0.999798059463501),
 ('Good', 0.9997888803482056),
 ('inoffensive', 0.999782145023346),
 ('staple', 0.9997768402099609),
 ('groundbreaking', 0.999772310256958),
 ('though', 0.9997702836990356),
 ('sometimes', 0.9997677206993103),
 ('going', 0.9997669458389282),
 ('bathroom', 0.9997662901878357),
 ('thought', 0.9997656941413879)]

In [ ]:
# https://projector.tensorflow.org/